# 🚀 AI Video Upscale & Interpolation Workflow

Welcome Huy! This notebook is designed to upscale your videos to 4K using **Real-ESRGAN** and optionally boost them to 60fps using **RIFE**.

### 📌 Workflow Overview:
1. **Connect Google Drive**
2. **Install Dependencies** (Real-ESRGAN & RIFE)
3. **Upscale to 4K** (Real-ESRGAN)
4. **Interpolate to 60fps** (RIFE)
5. **Utility tools** (Split/Merge/Check status)

## 1️⃣ Setup Environment
First, mount your Google Drive and set up the GPU.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Check GPU Status
!nvidia-smi

## 2️⃣ Install Tools
This installs Real-ESRGAN and RIFE repositories.

In [ ]:
# Install Real-ESRGAN
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
!pip install basicsr facexlib gfpgan
!pip install -r requirements.txt
!python setup.py develop

# Pre-download weights (Anime Video v3)
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.5.0/realesr-animevideov3.pth -P weights/

In [ ]:
# Install RIFE
%cd /content
!git clone https://github.com/hzwer/Practical-RIFE.git RIFE
%cd RIFE
!gdown --id 1wsQIhf9Y7v_z5V06rX8Nia62nswqLpS0
!7z e RIFE_trained_model_v4.7.zip

## 3️⃣ Run Upscale (Real-ESRGAN)
Adjust the input and output paths below.

In [ ]:
import os

INPUT_PATH = "/content/drive/MyDrive/Movies/Input/phim_cua_huy.mp4" # @param {type:"string"}
OUTPUT_PATH = "/content/drive/MyDrive/Movies/Output/phim_4K.mp4" # @param {type:"string"}
SCALE = 2 # @param {type:"integer"}
MODEL_NAME = "realesr-animevideov3" # @param ["realesr-animevideov3", "RealESRGAN_x4plus_anime_6B"]

%cd /content/Real-ESRGAN
!python inference_realesrgan_video.py \
    -n {MODEL_NAME} \
    -i {INPUT_PATH} \
    -o {OUTPUT_PATH} \
    -s {SCALE} \
    --suffix 4k

## 4️⃣ Run 60FPS Interpolation (RIFE)
Run this to make the video smoother.

In [ ]:
UPSCALED_FILE = "/content/drive/MyDrive/Movies/Output/phim_4K_4k.mp4" # @param {type:"string"}
FINAL_OUTPUT = "/content/drive/MyDrive/Movies/Output/phim_4K_60fps.mp4" # @param {type:"string"}
EXPONENT = 1 # @param {type:"integer"} 
# Exponent 1 creates 2x frames (24 -> 48 or 30 -> 60). Exponent 2 creates 4x frames.

%cd /content/RIFE
!python3 inference_video.py \
    --video={UPSCALED_FILE} \
    --output={FINAL_OUTPUT} \
    --exp={EXPONENT}

## 5️⃣ Utility: Split & Merge (FFmpeg)
If your video is too long, use these to process in chunks.

In [ ]:
# Split video into 30-minute chunks
!ffmpeg -i "/path/to/large_video.mp4" -f segment -segment_time 1800 -c copy part_%03d.mp4

In [ ]:
# Merge video parts (Create file_list.txt first)
# !ffmpeg -f concat -safe 0 -i file_list.txt -c copy merged_video.mp4